#  LoopAR Pilot Test

In [2]:

%matplotlib widget
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date
import json
import numpy as np
#print(os.getcwd())

### Defaults

In [3]:
ET_DATA_PATH = 'D:\_MasterArbeit_EEDA\EyeTracking'
SCENE_EVENT_DATA_PATH = 'D:\_MasterArbeit_EEDA\SceneData'
STEERING_DATA_PATH = 'D:\_MasterArbeit_EEDA\Input'
PARTICIPANT_DATA_PATH = 'D:\_MasterArbeit_EEDA\ParticipantCalibrationData'
PROCESSED_DATA_PATH = '../ProcessedData/'
PLOT_PATH = '../PLOTS/data_quality/'

os.makedirs(os.path.dirname(PROCESSED_DATA_PATH), exist_ok=True)
os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

## Eye Tracking Data Check

In [4]:
#Samples_df is of type DataFrame
et_samples_df = pd.DataFrame()

files = glob.glob(f'{ET_DATA_PATH}/*.txt')
#display(files)

for fi in files[2:3]: # reading only one file for now
    display(fi)
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
    for nS, sample in enumerate(tmp):
        #display(tmp)
        tmpdf = pd.json_normalize(data=sample, sep='_')
        #display(tmpdf)
        tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
        tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
        et_samples_df = pd.concat([et_samples_df, tmpdf], ignore_index=True)
    

et_samples_df['timestamp'] = pd.to_datetime(et_samples_df.UnixTimeStamp, utc=True, unit='s', origin='unix')

#pd.set_option('display.max_rows', samples_df.shape[0]+1)
#display(et_samples_df['timestamp'])

'D:\\_MasterArbeit_EEDA\\EyeTracking\\2e0a0e94ea7a477b93b7c44301bdc516_EyeTracking_MountainRoad.txt'

## Steeringwheel Data reading


In [5]:
steering_df = pd.DataFrame()

files = glob.glob(f'{STEERING_DATA_PATH}/*.txt')
# display(files)

for fi in files[2:3]: # reading only one file for now
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
    for nS, sample in enumerate(tmp):
        tmpdf = pd.json_normalize(data=sample, sep='_')
        tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
        tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
        steering_df = pd.concat([steering_df, tmpdf], ignore_index=True)
    

steering_df['timestamp'] = pd.to_datetime(steering_df.TimeStamp, utc=True, unit='s', origin='unix')


### Check if timestamps allign

In [6]:
#display(steering_df)
ne = pd.DataFrame()
i = 0
ne['Equal'] = np.where(steering_df['timestamp'] == et_samples_df['timestamp'],'1','0')
for index,row in ne.iterrows():
    if(row['Equal'])!='1':
        steering_df = steering_df.drop(labels=[row.name], axis=0)
        et_samples_df = et_samples_df.drop(labels=[row.name], axis=0)
        #print(row.name)        
        i = i+1

#i

In [7]:
# If one DF is to long
#steering_df.drop(steering_df.tail(1).index,inplace=True)
# fuse the two DF if euqal
et_samples_df = pd.merge(steering_df, et_samples_df)

## Callibration Participant Data

In [ ]:
#Samples_df is of type DataFrame
part_samples_df = pd.DataFrame()

files = glob.glob(f'{PARTICIPANT_DATA_PATH}/*.txt')
#display(files)

for fi in files[2:3]: # reading only one file for now
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
        tempdf0 = pd.DataFrame.from_dict(tmp,orient='index').T        
        tempdf0[["EyeValidationError.x","EyeValidationError.y","EyeValidationError.z"]] = pd.DataFrame(tempdf0["EyeValidationError"].to_list(), columns=['x', 'y','z'])
        tempdf0[["SeatCalibrationOffset.x","SeatCalibrationOffset.y","SeatCalibrationOffset.z"]] = pd.DataFrame(tempdf0["SeatCalibrationOffset"].to_list(), columns=['x', 'y','z'])
        part_samples_df = pd.concat([part_samples_df, tempdf0], ignore_index=True)
        
display(part_samples_df)

## Event Data Check

In [ ]:
event_samples_df = pd.DataFrame()
interims_df = pd.DataFrame()
Event_dur_df = pd.DataFrame()
#pd.set_option("display.max_colwidth", -1)

files = glob.glob(f'{SCENE_EVENT_DATA_PATH}/*.txt')
for fi in files[2:3]: # reading only one file for now
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
        x = tmp.get('EventBehavior')
        for i in range(len(x)):
            tmpdf = pd.json_normalize(data=x[i], sep='_')
            interims_df = tmpdf[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
       'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
            Event_dur_df = pd.concat([Event_dur_df,interims_df],ignore_index = True)
            tmpdf2 = pd.json_normalize(tmpdf["dynamicObjectData"])
            for nS, sample in enumerate(tmpdf2):
                temp = list(tmpdf2[nS])
                tmpdf3 = pd.json_normalize(data=temp, sep='_')
                tmpdf3['uid'] = fi.split('\\')[-1].split('_')[0]
                tmpdf3['scene'] = fi.split('_')[-1].split('.')[0]
                tmpdf3[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
       'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']] = interims_df[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
       'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                event_samples_df = pd.concat([event_samples_df, tmpdf3], ignore_index=True)
                
event_samples_df['timestamp'] = pd.to_datetime(event_samples_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
event_samples_df['StartofEventTime'] = pd.to_datetime(event_samples_df.StartofEventTimeStamp, utc=True, unit='s', origin='unix')
event_samples_df['EndofEventTime'] = pd.to_datetime(event_samples_df.EndOfEventTimeStamp, utc=True, unit='s', origin='unix')

#pd.set_option('display.max_columns', event_samples_df.shape[0]+1)
#display(event_samples_df)

### Get Event Time frames

In [ ]:
Start = et_samples_df.iloc[0]['timestamp']
#print(Start)
Event_dur_df['StartofEventTimeStamp'] = pd.to_datetime(Event_dur_df.StartofEventTimeStamp, utc=True, unit='s', origin='unix')
Event_dur_df['EndOfEventTimeStamp'] = pd.to_datetime(Event_dur_df.EndOfEventTimeStamp, utc=True, unit='s', origin='unix')
#Event_dur_df['StartofEventTimeStamp']
Event_dur_df['time_from_start_to_Event'] = ((Event_dur_df['StartofEventTimeStamp']- Start) /np.timedelta64(1,'s'))
Event_dur_df['time_from_start_to_End'] = ((Event_dur_df['EndOfEventTimeStamp']- Start) /np.timedelta64(1,'s'))
display(Event_dur_df)

In [ ]:
et_samples_df.scene.unique()
#event_samples_df.scene.unique()

In [ ]:
#display(et_samples_df[['EyeBlinkingWorldLeftTobii', 'EyeBliningWorldRightTobii',
#       'EyeBlinkingLocalLeftTobii', 'EyeBlinkingLocalRightTobii']])
#display(et_samples_df.loc())
np.where(et_samples_df[['EyeBlinkingWorldLeftTobii']])[1]

### eye velocity

In [8]:
# Calculate head & eye angles
# read up on arctan
et_samples_df.reset_index(inplace=False)
et_samples_df['eye_theta_h'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_x'], 
                                       et_samples_df['EyeDirectionCombinedLocal_z'])
et_samples_df['eye_theta_v'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_y'], 
                                       et_samples_df['EyeDirectionCombinedLocal_z'])

et_samples_df['head_theta_h'] = np.arctan2(et_samples_df['NoseVector_x'], 
                                        et_samples_df['NoseVector_z'])
et_samples_df['head_theta_v'] = np.arctan2(et_samples_df['NoseVector_y'],
                                        et_samples_df['NoseVector_z'])

# Calculate head & eye velocity
et_samples_df['eye_vel_h'] = (et_samples_df
                                .groupby(['uid', 'scene'])
                                .eye_theta_h
                                .apply(lambda x: x.diff())
                          )/(et_samples_df
                                .groupby(['uid', 'scene'])
                                .timestamp
                                .apply(lambda x: x.diff()/np.timedelta64(1, 's')))

et_samples_df['eye_vel_v'] = (et_samples_df
                                .groupby(['uid', 'scene'])
                                .eye_theta_v
                                .apply(lambda x: x.diff())
                          )/(et_samples_df
                                .groupby(['uid', 'scene'])
                                .timestamp
                                .apply(lambda x: x.diff()/np.timedelta64(1, 's')))

et_samples_df['head_vel_h'] = (et_samples_df
                                .groupby(['uid', 'scene'])
                                .head_theta_h
                                .apply(lambda x: x.diff())
                          )/(et_samples_df
                                .groupby(['uid', 'scene'])
                                .timestamp
                                .apply(lambda x: x.diff()/np.timedelta64(1, 's')))

et_samples_df['head_vel_v'] = (et_samples_df
                                .groupby(['uid', 'scene'])
                                .head_theta_v
                                .apply(lambda x: x.diff())
                          )/(et_samples_df
                                .groupby(['uid', 'scene'])
                                .timestamp
                                .apply(lambda x: x.diff()/np.timedelta64(1, 's')))


In [9]:
et_samples_df['eye_angular_vel'] =np.sqrt(et_samples_df.eye_vel_h**2 + et_samples_df.eye_vel_v**2)
et_samples_df['head_angular_vel'] =np.sqrt(et_samples_df.head_vel_h**2 + et_samples_df.head_vel_v**2)

In [10]:
et_samples_df.dropna(axis=0,subset=['eye_angular_vel', 'eye_vel_h', 'head_angular_vel', 'head_vel_h'], inplace=True)

In [11]:
et_samples_df.set_index('timestamp', inplace=True)

#create column which keeps track of time elapsed since trial start
et_samples_df['time_from_start'] = (et_samples_df
#                                    .reset_index()
                                   .groupby(['uid','scene',], as_index=False)
#                                    .timestamp
                                   .transform(lambda x: 
                                              (x.index - x.index[0])/np.timedelta64(1,'s'))
                                   .UnixTimeStamp
                                  )

#display(et_samples_df)

In [12]:
steering_df = et_samples_df
steering_df.columns
#steering_df['eye_vel_h'] = steering_df.eye_vel_h.div(300)

Index(['TimeStamp', 'ReceivedInput', 'SteeringInput', 'AcellerationInput',
       'BrakeInput', 'uid', 'scene', 'UnixTimeStamp', 'TobiiTimeStamp', 'FPS',
       'EyeOpennessLeftSranipal', 'EyeOpennessRightSranipal',
       'pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight',
       'EyeBlinkingWorldLeftTobii', 'EyeBliningWorldRightTobii',
       'EyeBlinkingLocalLeftTobii', 'EyeBlinkingLocalRightTobii', 'hitObjects',
       'HmdPosition_x', 'HmdPosition_y', 'HmdPosition_z', 'NoseVector_x',
       'NoseVector_y', 'NoseVector_z', 'HmdRotation_x', 'HmdRotation_y',
       'HmdRotation_z', 'HmdRotation_w', 'EyePositionCombinedWorld_x',
       'EyePositionCombinedWorld_y', 'EyePositionCombinedWorld_z',
       'EyeDirectionCombinedWorld_x', 'EyeDirectionCombinedWorld_y',
       'EyeDirectionCombinedWorld_z', 'EyePositionCombinedLocal_x',
       'EyePositionCombinedLocal_y', 'EyePositionCombinedLocal_z',
       'EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y',
       'E

## Steering data check

### Check if timestamps allign


In [ ]:
steering_df.set_index('timestamp', inplace=True)
#create column which keeps track of time elapsed since trial start
steering_df['time_from_start'] = (steering_df
#                                    .reset_index()
                                   .groupby(['uid','scene',], as_index=False)
#                                    .timestamp
                                   .transform(lambda x: 
                                              (x.index - x.index[0])/np.timedelta64(1,'s'))
                                   .TimeStamp
                                  )



In [ ]:
display(steering_df.columns)

In [ ]:
(
    steering_df
    .reset_index()
    .groupby('timestamp')
    .size()
    .plot(kind='hist')
)

In [13]:

for subjectID in steering_df.uid.unique():
    for scene in steering_df.scene.unique():
        tmpdf = steering_df.query('uid == @subjectID and scene == @scene')
        tmpdf = tmpdf.drop_duplicates(subset=['time_from_start'], keep='first')

        _, ax = plt.subplots(1, 1, figsize=(30,5), sharex=True, sharey=False)
#         ax = ax.ravel()
        (
            tmpdf
            .plot(
                x='time_from_start',y='SteeringInput', kind='line', color='k', lw=3 ,
                ax=ax, label='Steering', legend=True)
        )
        (
            tmpdf
            .plot(
                x='time_from_start',y='AcellerationInput', kind='line', color='g', 
                ax=ax, label='Acceleration', legend=True)
        )
        
        (
            tmpdf
            .plot(
                x='time_from_start',y='BrakeInput', kind='line', color='r', 
                ax=ax, label='Brake', legend=True)
        )
        (
            tmpdf
            .plot(
                x='time_from_start',y='eye_theta_h', kind='line', color='y', 
                ax=ax, label='Eyemovement-horicontal', legend=True)
        )
        
        
        #ax.set_ylabel("My ylabel")

        plt.suptitle(f'Scene: {scene}')
        plt.savefig(f'{PLOT_PATH}/driving_behavior_{scene}.png', dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …